# **Báo cáo bài tập lớn môn Học máy**
**Sinh viên:** Phan Hải Anh<br>
**MSSV:** 19021213<br>

# **Mô tả bài toán**

**Quora Insincere Question Classification** là một bài toán của **Quora** đặt ra, sử dụng sự trợ giúp từ cộng đồng, giúp họ phân loại những câu hỏi không chân thành. Nhiệm vụ của bài toán là sử dụng tập dữ liệu mà **Quora** cung cấp để phân loại ra những câu hỏi mang hàm ý không chân thành, mang nội dung xấu độc, gây hiểu lầm.

- **Đầu vào**: Câu hỏi dưới dạng văn bản (plain text)
- **Đầu ra**: Yes/No (insincere (1) or not (0))

# **Hướng Tiếp Cận**
- Tiền xử lý bộ dữ liệu
    - Bỏ đi các biểu thức toán học và các đường dẫn liên kết
    - Bỏ đi các ký tự đặc biệt, các chữ số
    - Sửa những từ sai chính tả và extend các từ viết tắt
    - Bỏ đi stop_words trong câu
    - Lemming các từ có trong câu (có thể hiểu là chuyển các từ về dạng nguyên thể)
    
> **Mục đích**: Loại bỏ đi các từ gây nhiễu cho bộ dữ liệu và mở rộng độ bao phủ của tập Vocab với bộ dữ liệu
- Xây dựng tập vocab: 
    - Sử dụng các file trong embeddings.zip mà quora cung cấp để xây dựng tập vocab 
    - Các file embeddings có dạng text, mỗi dòng chứa 1 word và đi kèm đó là vector của nó đã được pretrained
    - Xây tập vocab từ các word có trong từ điển và vector đi kèm với chúng
    - Kiểm tra độ phủ của vocab xây dựng dựa trên tập embeddings với dữ liệu được cho
- Xây dựng ma trận embeddings:
    - Sử dụng file pretrained để tạo ra ma trận embeddings dựa vào các từ có trong tập vocab
    - Lấy các từ để lookup vector descriptor của các từ đó và xây ma trận
- Tạo model và huấn luyện dự đoán
- Kết hợp kết quả của nhiều model với các ma trận embeddings đã xây dựng và cho ra kết quả tốt nhất

# **1. Khảo sát dữ liệu**

In [ ]:
import pandas as pd 
import seaborn as sns
import re
import gc
import os
import numpy as np
import operator
from wordcloud import WordCloud, STOPWORDS
from tqdm import tqdm
import matplotlib.pyplot as plt
pd_ctx = pd.option_context('display.max_colwidth', 100)

import nltk
from nltk.stem import PorterStemmer, SnowballStemmer, WordNetLemmatizer

from gensim.models import KeyedVectors

import tensorflow as tf

from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D,GRU
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, SpatialDropout1D, GlobalMaxPooling1D, Concatenate
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences

Xác định các tập ```train```, ```test``` và khởi tạo một tập chứa dữ liệu của cả 2 để về sau check độ bao phủ của vocab với tập dữ liệu đã sinh

In [ ]:
df_train = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
df_test = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
quora_data = df_train['question_text'].append(df_test['question_text'])

### **a. Khảo sát dữ liệu trong tập **train****

In [ ]:
print("\033[1mTrain set info\033[0m")
print(df_train.info())

Ở trong bộ dữ liệu có 3 trường, và khả năng dữ liệu train nằm ở trường **```question_text```** <br>
Vậy ta sẽ thử khảo sát dữ liệu trong trường **```question_text```**

In [ ]:
# Kiểm tra các trường dữ liệu của câu hỏi được đánh sincere
print("\033[1mSincere Questions: \033[0m")
display(df_train[df_train['target']==0].head())
# Kiểm tra các trường dữ liệu của câu hỏi được đánh sincere
print("\033[1mInsincere Questions: \033[0m")
display(df_train[df_train['target']==1].head())

Có thể thấy rằng tập train có 3 trường dữ liệu
- **```qid```**: định danh id cho câu hỏi
- **```question_text```**: nội dung câu hỏi
- **```target```**: phân lớp câu hỏi
    - 0: câu hỏi mang tính chất chân thành (sincere)
    - 1: câu hỏi không mang tính chất chân thành (insincere)
    
Khá may mắn khi tập dữ liệu **Quora** cung cấp không có bất kì trường dữ liệu nào có giá trị bất thường (```null```, ```none```, ```missing```)
    
Tập dữ liệu train bao gồm **1306122** dòng dữ liệu, gần **1.31 triệu dòng**, khá lớn. Cần sử dụng mô hình có hiệu quả tính toán nhanh, nên trong bài này em sẽ không sử dụng mô hình với thuật toán ensemble

Theo dự đoán thì trường **```target```** sẽ là đánh dấu các câu hỏi sincere và insincere nên cần xem thử là trong trường **```target```** đó có những giá trị gì

In [ ]:
df_train.target.value_counts()

Có thể thấy trong trường target có 2 giá trị là ```0``` và ```1```<br>
Dự đoán thì những câu hỏi đánh dấu ```0``` là những câu hỏi sincere, tương tự ```1``` là insincere<br>
Vậy có thể xem như đây là bài toán phân lớp nhị phân<br>
Nhưng vẫn cần phải có đánh giá sâu hơn về bộ dữ liệu này, nhìn nhanh qua có thể thấy bộ dữ liệu train được cho khá là mất cân bằng

In [ ]:
pos_len = len(df_train[df_train['target'] == 1])
neg_len = len(df_train[df_train['target'] == 0])
total = len(df_train)
print("\033[1mTotal = \033[0m", total)
print("\033[1mSincere questions:\033[0m {neg} ({percent: .2f}% )".format(neg = neg_len, percent = neg_len / total * 100))
print("\033[1mInsincere questions:\033[0m {pos} ({percent: .2f}% )".format(pos = pos_len, percent = pos_len / total * 100))
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(['sincere', 'insincere'], df_train.target.value_counts())
plt.show()

Qua biểu đồ có thể dễ dàng nhận thấy tập dữ liệu train bị mất cân bằng nhiều:
- Lớp câu hỏi sincere có **1225312** (chiếm **93.81%**) số lượng dữ liệu
- Lớp câu hỏi insincere có **80810** (chiếm **6.19%**) số lượng dữ liệu

Có thể thấy tập dữ liệu bị lệch nhiều (lên tới **15 lần**) cho nên cần phải sử dụng metrics f1_score để đánh giá độ hiệu quả của mô hình hơn là so **accuracy**<br>
Do **f1_score** quan tâm đến phân bố của dữ liệu nên trong bài này nó sẽ mang nhiều ý nghĩa hơn là **accuracy**.<br>
Và trong cuộc thi **Kaggle** cũng bảo là sử dụng **f1_score** thay vì **accuracy**

F1 score là sự **harmonic mean** của **Precision** và **Recall**. F1 Score nhận giá trị trong khoảng ```0``` - ```1```<br>
$$Precision = \frac{TP}{TP+FP}$$
$$Recall = \frac{TP}{TP+FN}$$
$$F1 = \frac{2}{Recall^{-1} + Precision^{-1}}$$

**Nhận xét:**<br>
Có thể coi đây là bài toán phân loại nhị phân. Thường thì ta sẽ nghĩ đến các mô hình tuyến tính như **Logistic Regression** hay **SVM** để phân loại. Nhưng đây là bài toán thiên về tiền xử lý do đầu vào là chuỗi các text, nên quan trọng là việc mình xử lý chuỗi như thế nào. Trong bài này em sẽ sử dụng thằng **Keras** với **LSTM Model** và **Embedings** các chuỗi text để giảm chiều dữ liệu

### **b. Khảo sát dữ liệu trong tập **test**** 

In [ ]:
df_test.info()

Ở trong bộ dữ liệu test thì chỉ có 2 trường dữ liệu là **```qid```** đại diện cho id của câu hỏi<br>
Và **```question_text```** đại diện cho dữ liệu test<br>
Cũng may mắn khi mà ở trong tập **df_test** lại không có dữ liệu kì lạ nào cả (```null```, ```none```, ```missing```)<br>
Và cái ta cần quan tâm ở đây chỉ là trường dữ liệu **```question_text```** để đưa vào model

In [ ]:
# Shuffle tập train để kiểm tra những giá trị ngẫu nhiên
train = df_train.sample(frac=1).reset_index(drop=True)
display(train.sample(n=10, random_state=344))

Mỗi lần shuffle lại cho ra một kết quả khác, nhưng nhìn chung qua vài lần thực nghiệm quan sát, em nhận thấy trong tập dữ liệu ở trên:
- Có nhiều câu mà các từ trong đó viết sai chính tả, lẫn lộn giữa tiếng Anh-Anh và tiếng Anh-Mỹ
- Nhiều câu sử dụng các từ viết tắt: He's mà đáng nhẽ nên là He is.
- Có một số từ viết tắt cho tên các tổ chức, hay các chuẩn mã hoá kiểu RSA, DES/3DES, ...
- Nhiều câu sử dụng các ký tự đặc biệt cho nên sẽ gây ảnh hưởng lớn tới mô hình dự đoán

Về mặt nội dung hàm ý thì có thể không ảnh hưởng nhưng sẽ ảnh hưởng khi xâp tập từ điển, cùng một ý nghĩa nhưng các từ lại được tính nhiều lần giá trị

> **Thấy rằng, bộ dữ liệu đã cho có khá nhiều nhiễu => cần loại bỏ nhiều nhiều nhất có thể**

**Cách xử lý**
- Loại bỏ stopwords có trong câu
- Stem các từ có trong câu (driver, driving, driven, drove -> drive)
- Chuẩn hoá các từ về dạng lowercase
- Loại bỏ các ký tự đặc biệt
- Loại bỏ các đường dẫn liên kết, loại bỏ các công thức toán học, ...

### **c. Nhận xét**
Nhưng có thể thấy một điều là bộ dữ liệu **train** và **test** dữ liệu đầu vào đều ở dạng plain text và mô hình của chúng ta sẽ không thể hiểu nổi<br>
Vậy hướng giải quyết ở đây em nghĩ là là sử dụng mã hoá **one_hot** đưa dữ liệu về dạng binary để cho mô hình có thể hiểu được

# **2. Tiền xử lý dữ liệu**

Với bài toán liên quan đến ngôn ngữ thì hiệu quả của mô hình phụ thuộc lớn vào việc tiền xử lý dữ liệu, tức loại bỏ đi nhiễu ở trong bộ dữ liệu.<br>
Và phương pháp phổ biến đối với các bài toán liên quan đến ngôn ngữ được trình bày ở dưới như sau

#### **Reference**: https://www.kaggle.com/canming/ensemble-mean-iii-64-36
**Cách làm:**<br>
Sử dụng thư viện xử lý ngôn ngữ tự nhiên nltk trong việc preprocess data<br>
Ở đây yêu cần phải sử dụng wordnet, một thư viện từ đồng nghĩa, trái nghĩa, nltk punkt được yêu cầu để tokenize words<br>
- Bước 1: ```clean_tag()```: loại bỏ đi các biểu thức toán học, các địa chỉ liên kết
- Bước 2: ```clean_puncts()```: loại bỏ đi các ký tự đặc biệt có trong câu
- Bước 3: ```correct_misspell()```: trong bộ dữ liệu của **Quora** do đây là các câu hỏi của người dùng nên không tránh khỏi việc gõ sai, hay là có những từ không chuẩn ví dụ là lẫn lộn giữa tiếng Anh-Anh với tiếng Anh-Mỹ cho nên cần phải fix những từ này và đưa nó về dạng chuẩn
- Bước 4: ```remove_stopwords()```: loại bỏ đi các stopwords có trong câu
- Bước 5: ```clean_contractions()```: chuyển những từ viết tắt về dạng đầy đủ vốn có
- Bước 6: ```lemming_words()```: lemming các từ về dạng nguyên bản của nó


**Vấn đề:**<br>
Tuy nhiên đời không như là mơ, cuộc thi này của **Quora** không cho phép sử dụng Internet nên việc lemming word bằng cách lookup từ điển này sẽ không hoạt động<br>
**Giải pháp:**<br>
Cần tìm ra một phương pháp lemming words mới

### **a. Định nghĩa các hàm xử lý**

In [ ]:
### do việc lemming words yêu cầu sử dụng thư viện wordnet để lookup các từ có trong đó
# nltk.download('wordnet')
# nltk.download('punkt')

Ở trong hàm **```clean_tag()```** em sẽ loại bỏ bằng các biểu thức toán học, và nếu có liên kết trong câu thì cũng sẽ loại bỏ đi và thay thế bằng URL<br>
Do các biểu thức toán học và đường dẫn liên kết không có nhiều giá trị về mặt ý nghĩa trong phân loại câu học sincere hay không sincere mà lại làm nhiễu bộ dữ liệu khá nhiều

In [ ]:
def clean_tag(x):
  if '[math]' in x:
    x = re.sub('\[math\].*?math\]', 'MATH EQUATION', x) #replacing with [MATH EQUATION]
    
  if 'http' in x or 'www' in x:
    x = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', 'URL', x) #replacing with [url]
  return x

Trong hàm **```clean_puncts()```** này thì em sẽ loại bỏ đi các kí tự đặc biệt và các emoji cảm xúc<br>
Bởi vì việc loại bỏ emoji sẽ không làm ảnh hưởng nhiều tới ý nghĩa của câu, tương tự như các dấu câu và kí tự đặc biệt<br>
Cũng như ý ở trên, có quá nhiều emoji và ký tự đặc biệt làm nhiễu bộ dữ liệu khá nhiều

In [ ]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

def clean_punct(x):
  x = str(x)
  for punct in puncts:
    if punct in x:
      x = x.replace(punct, ' ')
    return x

Sửa những từ dễ gây hiểu lầm **```correct_misspell()```**, ở trong mảng định nghĩa sẵn thì em sẽ convert tiếng Anh-Anh sang tiếng Anh-Mỹ, bên cạnh đó sửa sai cho những từ dễ viết sai<br>
Bởi vì Quora fetch các câu hỏi của người dùng nên việc sai chính tả là không thể tránh khỏi<br>
Mục đích là để convert sang càng nhiều từ càng tốt, qua đó đạt được độ phủ cao với vocab được build từ tệp GoogleNews đính kèm trong file ```embeddings.zip```

In [ ]:
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'bitcoin', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization', 
                'electroneum':'bitcoin','nanodegree':'degree','hotstar':'star','dream11':'dream','ftre':'fire','tensorflow':'framework','unocoin':'bitcoin',
                'lnmiit':'limit','unacademy':'academy','altcoin':'bitcoin','altcoins':'bitcoin','litecoin':'bitcoin','coinbase':'bitcoin','cryptocurency':'cryptocurrency',
                'simpliv':'simple','quoras':'quora','schizoids':'psychopath','remainers':'remainder','twinflame':'soulmate','quorans':'quora','brexit':'demonetized',
                'iiest':'institute','dceu':'comics','pessat':'exam','uceed':'college','bhakts':'devotee','boruto':'anime',
                'cryptocoin':'bitcoin','blockchains':'blockchain','fiancee':'fiance','redmi':'smartphone','oneplus':'smartphone','qoura':'quora','deepmind':'framework','ryzen':'cpu','whattsapp':'whatsapp',
                'undertale':'adventure','zenfone':'smartphone','cryptocurencies':'cryptocurrencies','koinex':'bitcoin','zebpay':'bitcoin','binance':'bitcoin','whtsapp':'whatsapp',
                'reactjs':'framework','bittrex':'bitcoin','bitconnect':'bitcoin','bitfinex':'bitcoin','yourquote':'your quote','whyis':'why is','jiophone':'smartphone',
                'dogecoin':'bitcoin','onecoin':'bitcoin','poloniex':'bitcoin','7700k':'cpu','angular2':'framework','segwit2x':'bitcoin','hashflare':'bitcoin','940mx':'gpu',
                'openai':'framework','hashflare':'bitcoin','1050ti':'gpu','nearbuy':'near buy','freebitco':'bitcoin','antminer':'bitcoin','filecoin':'bitcoin','whatapp':'whatsapp',
                'empowr':'empower','1080ti':'gpu','crytocurrency':'cryptocurrency','8700k':'cpu','whatsaap':'whatsapp','g4560':'cpu','payymoney':'pay money',
                'fuckboys':'fuck boys','intenship':'internship','zcash':'bitcoin','demonatisation':'demonetization','narcicist':'narcissist','mastuburation':'masturbation',
                'trignometric':'trigonometric','cryptocurreny':'cryptocurrency','howdid':'how did','crytocurrencies':'cryptocurrencies','phycopath':'psychopath',
                'bytecoin':'bitcoin','possesiveness':'possessiveness','scollege':'college','humanties':'humanities','altacoin':'bitcoin','demonitised':'demonetized',
                'brasília':'brazilia','accolite':'accolyte','econimics':'economics','varrier':'warrier','quroa':'quora','statergy':'strategy','langague':'language',
                'splatoon':'game','7600k':'cpu','gate2018':'gate 2018','in2018':'in 2018','narcassist':'narcissist','jiocoin':'bitcoin','hnlu':'hulu','7300hq':'cpu',
                'weatern':'western','interledger':'blockchain','deplation':'deflation', 'cryptocurrencies':'cryptocurrency', 'bitcoin':'blockchain cryptocurrency',}

def correct_mispell(x):
  words = x.split()
  for i in range(0, len(words)):
    if mispell_dict.get(words[i]) is not None:
      words[i] = mispell_dict.get(words[i])
    elif mispell_dict.get(words[i].lower()) is not None:
      words[i] = mispell_dict.get(words[i].lower())
        
  words = " ".join(words)
  return words

**```remove_stopwords()```**: tức là những từ, chữ dạng *'do'*, *'does'*, *'did'*, *'should'*, ...<br>
Tuy nhiên qua lần chạy mà không loại bỏ stopwords thì kết quả thu nhận lại khả thi hơn với f1_score đạt trung bình ```0.64```<br>
Với lần chạy mà loại bỏ stopwords thì kết quả thu nhận lại được với f1_score loanh quanh ```0.63x``` (x học tiểu học) và cao nhất là ```0.633``` qua nhiều lần thử<br>
- Qua đó có thể đưa ra kết luận rằng việc loại bỏ stopwords trong mô hình em xây dựng ở bài này đã làm mất đi một phần ý nghĩa của câu và qua đó giảm độ chính xác của mô hình
- Có vẻ như việc loại bỏ stop_words đã gián tiếp loại bỏ đi những từ có trong vocab được build từ GoogleNews và làm giảm độ phủ của vocab đối với dữ liệu được xử lý

In [ ]:
def remove_stopwords(x):
  x = [word for word in x.split() if word not in STOPWORDS]
  x = ' '.join(x)
  return x

**```clean_contractions()```**: Map những từ viết tắt sang dạng hoàn chỉnh, tránh sự hiểu lầm và không làm mất đi ý nghĩa của câu hỏi<br>
Mục đích là để đạt được độ phủ tốt nhất của vocab được build từ file embeddings

In [ ]:
contraction_mapping = {
 "I'm": 'I am',
 "I'm'a": 'I am about to',
 "I'm'o": 'I am going to',
 "I've": 'I have',
 "I'll": 'I will',
 "I'll've": 'I will have',
 "I'd": 'I would',
 "I'd've": 'I would have',
 "i'm": 'i am',
 "i'm'a": 'i am about to',
 "i'm'o": 'i am going to',
 "i've": 'i have',
 "i'll": 'i will',
 "i'll've": 'i will have',
 "i'd": 'i would',
 "i'd've": 'i would have',
 'Whatcha': 'What are you',
 'whatcha': 'what are you',
 "amn't": 'am not',
 "ain't": 'are not',
 "aren't": 'are not',
 "'cause": 'because',
 "can't": 'can not',
 "can't've": 'can not have',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "daren't": 'dare not',
 "daresn't": 'dare not',
 "dasn't": 'dare not',
 "didn't": 'did not',
 'didn’t': 'did not',
 "don't": 'do not',
 'don’t': 'do not',
 "doesn't": 'does not',
 "e'er": 'ever',
 "everyone's": 'everyone is',
 'finna': 'fixing to',
 'gimme': 'give me',
 "gon't": 'go not',
 'gonna': 'going to',
 'gotta': 'got to',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he've": 'he have',
 "he's": 'he is',
 "he'll": 'he will',
 "he'll've": 'he will have',
 "he'd": 'he would',
 "he'd've": 'he would have',
 "here's": 'here is',
 "how're": 'how are',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how's": 'how is',
 "how'll": 'how will',
 "isn't": 'is not',
 "it's": 'it is',
 "'tis": 'it is',
 "'twas": 'it was',
 "it'll": 'it will',
 "it'll've": 'it will have',
 "it'd": 'it would',
 "it'd've": 'it would have',
 'kinda': 'kind of',
 "let's": 'let us',
 'luv': 'love',
 "ma'am": 'madam',
 "may've": 'may have',
 "mayn't": 'may not',
 "might've": 'might have',
 "mightn't": 'might not',
 "mightn't've": 'might not have',
 "must've": 'must have',
 "mustn't": 'must not',
 "mustn't've": 'must not have',
 "needn't": 'need not',
 "needn't've": 'need not have',
 "ne'er": 'never',
 "o'": 'of',
 "o'clock": 'of the clock',
 "ol'": 'old',
 "oughtn't": 'ought not',
 "oughtn't've": 'ought not have',
 "o'er": 'over',
 "shan't": 'shall not',
 "sha'n't": 'shall not',
 "shalln't": 'shall not',
 "shan't've": 'shall not have',
 "she's": 'she is',
 "she'll": 'she will',
 "she'd": 'she would',
 "she'd've": 'she would have',
 "should've": 'should have',
 "shouldn't": 'should not',
 "shouldn't've": 'should not have',
 "so've": 'so have',
 "so's": 'so is',
 "somebody's": 'somebody is',
 "someone's": 'someone is',
 "something's": 'something is',
 'sux': 'sucks',
 "that're": 'that are',
 "that's": 'that is',
 "that'll": 'that will',
 "that'd": 'that would',
 "that'd've": 'that would have',
 'em': 'them',
 "there're": 'there are',
 "there's": 'there is',
 "there'll": 'there will',
 "there'd": 'there would',
 "there'd've": 'there would have',
 "these're": 'these are',
 "they're": 'they are',
 "they've": 'they have',
 "they'll": 'they will',
 "they'll've": 'they will have',
 "they'd": 'they would',
 "they'd've": 'they would have',
 "this's": 'this is',
 "those're": 'those are',
 "to've": 'to have',
 'wanna': 'want to',
 "wasn't": 'was not',
 "we're": 'we are',
 "we've": 'we have',
 "we'll": 'we will',
 "we'll've": 'we will have',
 "we'd": 'we would',
 "we'd've": 'we would have',
 "weren't": 'were not',
 "what're": 'what are',
 "what'd": 'what did',
 "what've": 'what have',
 "what's": 'what is',
 "what'll": 'what will',
 "what'll've": 'what will have',
 "when've": 'when have',
 "when's": 'when is',
 "where're": 'where are',
 "where'd": 'where did',
 "where've": 'where have',
 "where's": 'where is',
 "which's": 'which is',
 "who're": 'who are',
 "who've": 'who have',
 "who's": 'who is',
 "who'll": 'who will',
 "who'll've": 'who will have',
 "who'd": 'who would',
 "who'd've": 'who would have',
 "why're": 'why are',
 "why'd": 'why did',
 "why've": 'why have',
 "why's": 'why is',
 "will've": 'will have',
 "won't": 'will not',
 "won't've": 'will not have',
 "would've": 'would have',
 "wouldn't": 'would not',
 "wouldn't've": 'would not have',
 "y'all": 'you all',
 "y'all're": 'you all are',
 "y'all've": 'you all have',
 "y'all'd": 'you all would',
 "y'all'd've": 'you all would have',
 "you're": 'you are',
 "you've": 'you have',
 "you'll've": 'you shall have',
 "you'll": 'you will',
 "you'd": 'you would',
 "you'd've": 'you would have',
 'jan.': 'january',
 'feb.': 'february',
 'mar.': 'march',
 'apr.': 'april',
 'jun.': 'june',
 'jul.': 'july',
 'aug.': 'august',
 'sep.': 'september',
 'oct.': 'october',
 'nov.': 'november',
 'dec.': 'december',
 'I’m': 'I am',
 'I’m’a': 'I am about to',
 'I’m’o': 'I am going to',
 'I’ve': 'I have',
 'I’ll': 'I will',
 'I’ll’ve': 'I will have',
 'I’d': 'I would',
 'I’d’ve': 'I would have',
 'i’m': 'i am',
 'i’m’a': 'i am about to',
 'i’m’o': 'i am going to',
 'i’ve': 'i have',
 'i’ll': 'i will',
 'i’ll’ve': 'i will have',
 'i’d': 'i would',
 'i’d’ve': 'i would have',
 'amn’t': 'am not',
 'ain’t': 'are not',
 'aren’t': 'are not',
 '’cause': 'because',
 'can’t': 'can not',
 'can’t’ve': 'can not have',
 'could’ve': 'could have',
 'couldn’t': 'could not',
 'couldn’t’ve': 'could not have',
 'daren’t': 'dare not',
 'daresn’t': 'dare not',
 'dasn’t': 'dare not',
 'doesn’t': 'does not',
 'e’er': 'ever',
 'everyone’s': 'everyone is',
 'gon’t': 'go not',
 'hadn’t': 'had not',
 'hadn’t’ve': 'had not have',
 'hasn’t': 'has not',
 'haven’t': 'have not',
 'he’ve': 'he have',
 'he’s': 'he is',
 'he’ll': 'he will',
 'he’ll’ve': 'he will have',
 'he’d': 'he would',
 'he’d’ve': 'he would have',
 'here’s': 'here is',
 'how’re': 'how are',
 'how’d': 'how did',
 'how’d’y': 'how do you',
 'how’s': 'how is',
 'how’ll': 'how will',
 'isn’t': 'is not',
 'it’s': 'it is',
 '’tis': 'it is',
 '’twas': 'it was',
 'it’ll': 'it will',
 'it’ll’ve': 'it will have',
 'it’d': 'it would',
 'it’d’ve': 'it would have',
 'let’s': 'let us',
 'ma’am': 'madam',
 'may’ve': 'may have',
 'mayn’t': 'may not',
 'might’ve': 'might have',
 'mightn’t': 'might not',
 'mightn’t’ve': 'might not have',
 'must’ve': 'must have',
 'mustn’t': 'must not',
 'mustn’t’ve': 'must not have',
 'needn’t': 'need not',
 'needn’t’ve': 'need not have',
 'ne’er': 'never',
 'o’': 'of',
 'o’clock': 'of the clock',
 'ol’': 'old',
 'oughtn’t': 'ought not',
 'oughtn’t’ve': 'ought not have',
 'o’er': 'over',
 'shan’t': 'shall not',
 'sha’n’t': 'shall not',
 'shalln’t': 'shall not',
 'shan’t’ve': 'shall not have',
 'she’s': 'she is',
 'she’ll': 'she will',
 'she’d': 'she would',
 'she’d’ve': 'she would have',
 'should’ve': 'should have',
 'shouldn’t': 'should not',
 'shouldn’t’ve': 'should not have',
 'so’ve': 'so have',
 'so’s': 'so is',
 'somebody’s': 'somebody is',
 'someone’s': 'someone is',
 'something’s': 'something is',
 'that’re': 'that are',
 'that’s': 'that is',
 'that’ll': 'that will',
 'that’d': 'that would',
 'that’d’ve': 'that would have',
 'there’re': 'there are',
 'there’s': 'there is',
 'there’ll': 'there will',
 'there’d': 'there would',
 'there’d’ve': 'there would have',
 'these’re': 'these are',
 'they’re': 'they are',
 'they’ve': 'they have',
 'they’ll': 'they will',
 'they’ll’ve': 'they will have',
 'they’d': 'they would',
 'they’d’ve': 'they would have',
 'this’s': 'this is',
 'those’re': 'those are',
 'to’ve': 'to have',
 'wasn’t': 'was not',
 'we’re': 'we are',
 'we’ve': 'we have',
 'we’ll': 'we will',
 'we’ll’ve': 'we will have',
 'we’d': 'we would',
 'we’d’ve': 'we would have',
 'weren’t': 'were not',
 'what’re': 'what are',
 'what’d': 'what did',
 'what’ve': 'what have',
 'what’s': 'what is',
 'what’ll': 'what will',
 'what’ll’ve': 'what will have',
 'when’ve': 'when have',
 'when’s': 'when is',
 'where’re': 'where are',
 'where’d': 'where did',
 'where’ve': 'where have',
 'where’s': 'where is',
 'which’s': 'which is',
 'who’re': 'who are',
 'who’ve': 'who have',
 'who’s': 'who is',
 'who’ll': 'who will',
 'who’ll’ve': 'who will have',
 'who’d': 'who would',
 'who’d’ve': 'who would have',
 'why’re': 'why are',
 'why’d': 'why did',
 'why’ve': 'why have',
 'why’s': 'why is',
 'will’ve': 'will have',
 'won’t': 'will not',
 'won’t’ve': 'will not have',
 'would’ve': 'would have',
 'wouldn’t': 'would not',
 'wouldn’t’ve': 'would not have',
 'y’all': 'you all',
 'y’all’re': 'you all are',
 'y’all’ve': 'you all have',
 'y’all’d': 'you all would',
 'y’all’d’ve': 'you all would have',
 'you’re': 'you are',
 'you’ve': 'you have',
 'you’ll’ve': 'you shall have',
 'you’ll': 'you will',
 'you’d': 'you would',
 'you’d’ve': 'you would have'
}

def clean_contractions(text):
#     text = text.lower()
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return text

**```lemma_text()```**: để loại bỏ những từ có các digit cuối kiểu s/es/ed mà không làm mất đi nhiều ý nghĩa của câu (eg: words -> word)
Việc sử dụng lemming thay vì stemming sẽ phần nào cải thiện độ chính xác của mô hình do:
- **Stemming** cắt đi phần kí tự đặc biệt cuối từ một cách máy móc
- **Lemming** thì lại thông qua từ đó, look up trong bảng từ vựng được thiết kế trước và thay thế vào chỗ đó
Do vậy thì tuy lemming yêu cầu thời gian xử lý lâu hơn, nhưng đem lại giá trị cao hơn về độ chính xác của mô hình

In [ ]:
lemmatizer = WordNetLemmatizer()
def lemma_text(x):
    x = x.split()
    x = [lemmatizer.lemmatize(word) for word in x]
    x = ' '.join(x)
    return x

Merge các hàm tiền xử lý vào một hàm duy nhất để thao tác

In [ ]:
def data_cleaning(x):
  x = clean_tag(x)
  x = clean_punct(x)
  x = correct_mispell(x)
  x = remove_stopwords(x)
  x = clean_contractions(x)
  x = lemma_text(x)
  return x

Ở đây em sẽ tạo ra một hàm tiền xử lý khác với mục đích giữ lại nhiều từ nhất có thể, nhằm đạt được độ phủ cao với file pretrained<br>
Ở trong hàm này, các công việc bao gồm:
- ```clean_contractions()```: chuyển những từ viết tắt về dạng đầy đủ vốn có
- loại bỏ đi các số trong câu

Ở trong hàm này em tránh việc loại bỏ đi các từ như ở hàm trên (```remove_stopwords()```, ```clean_tag()```, ```clean_puncts()```, ```correct_mispell()```)<br>
Và em sẽ không convert words sang dạng ```lowercase```

In [ ]:
def Preprocess(doc):
    corpus=[]
    for text in tqdm(doc):
        text=clean_contractions(text)
        text=correct_mispell(text)
        text=re.sub(r'[^a-z0-9A-Z]'," ",text)
        text=re.sub(r'[0-9]{1}',"#",text)
        text=re.sub(r'[0-9]{2}','##',text)
        text=re.sub(r'[0-9]{3}','###',text)
        text=re.sub(r'[0-9]{4}','####',text)
        text=re.sub(r'[0-9]{5,}','#####',text)
        corpus.append(text)
    return corpus

### **b. Tiền xử lý các tập dữ liệu**
Công đoạn tiền xử lý khá nhanh khi tốn trung bình là 30 giây

In [ ]:
tqdm.pandas(desc="progress-bar")
df_test['question_text_cleaned'] = df_test['question_text'].progress_map(lambda x: data_cleaning(x))
df_train['question_text_cleaned'] = df_train['question_text'].progress_map(lambda x: data_cleaning(x))

Kiểm tra lại xem các hàm có hoạt động đúng như kỳ vọng không

In [ ]:
df_train.head(5)

Có thể thấy việc loại bỏ nhiễu theo hàm tiền xử lý ```data_clean()``` đã hoạt động tốt, chuẩn hoá các câu một cách chính xác<br>
Tuy nhiên lại gây một trở ngại là nó làm mất đi khá nhiều từ, có thể nó không quá quan trọng nhưng ít nhiều cũng mang một phần ý nghĩa<br>
Ta sẽ so sánh hiệu năng giữa 2 hàm tiền xử lý ở bước sau, khi ta build tập ```vocabulary``` và kiểm tra độ phủ đối với tập dữ liệu được xử lý

# **3. Build tập vocab**

Ở đây em sẽ cố định sử dụng thằng **Google News** làm tiêu chuẩn<br>
Kiểm tra word embeddings đã giải nén với **Google News** đính kèm trong file embeddings.zip<br>
Định nghĩa các hàm để build vocab từ file **Google News** và hàm kiểm tra độ bao phủ của vocab đối với tập dữ liệu đã được preprocess<br>
&nbsp;<br>
Trong bài này, em sẽ giải nén tệp embedding để sử dụng tập các vector với các từ cho sẵn<br>
Các từ trong file embeddings đều đính kèm một vector, và em sẽ xây ma trận embeddings dựa vào file embeddings đó

In [ ]:
%%time
### unzipping all the pretrained embeddings
!unzip ../input/quora-insincere-questions-classification/embeddings.zip

In [ ]:
!du -h ./

File **Google News** có kích thước khá nhỏ: ```3.4G``` nhưng nó lại ở dạng binary của word embeddings nên đã có thư viện KeyedVector hỗ trợ và việc load khá nhanh nên em sẽ sử dụng thằng **Google News** này làm tham chiếu cho các embeddings khác trong tương lai<br>
Ở dưới em sẽ định nghĩa các hàm build ```vocab``` và check độ phủ<br>
Do embeddings là file có word đính kèm vector mô tả cho nó nên em sẽ lookup các từ có trong câu với file embeddings và tìm vector của chúng

In [ ]:
%%time

### Loading tập Google News Pretrained Embeddings vào bộ nhớ
file_name="./GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
model_embed=KeyedVectors.load_word2vec_format(file_name,binary=True)
# model_embed = load_embed1('./glove.840B.300d/glove.840B.300d.txt')

### Xây dựng tập từ vựng dựa trên dữ liệu của tập Google News
def vocab_build(corpus):
    vocab={}
    for text in tqdm(corpus):
        for word in text.split():
            try:
                vocab[word]+=1
            except KeyError:
                vocab[word]=1
    return vocab


### Kiểm tra tập Vocabulary xem tập vocab đó bao phủ bao nhiêu phần trăm tập dữ liệu của mình
def check_voc(vocab,model):
    embed_words=[]
    out_vocab={}
    total_words=0
    total_text=0
    for i in tqdm(vocab):
        try:
            vec=model[i]
            embed_words.append(vec)
            total_words+=vocab[i]
        except KeyError:
            out_vocab[i]=vocab[i]
            total_text+=vocab[i]
    print("The {:.2f}% of vocabularies have Covered of corpus".format(100*len(embed_words)/len(vocab)))
    print("The {:.2f}% of total text had coverded ".format((100*total_words/(total_words+total_text))))
    return out_vocab

Xây dựng tập vocab và kiểm tra độ bao phủ của tập vocab với tập vocab của pretrained model từ Google News<br>
Trong block code này em sẽ xây tập vocab dựa trên bộ dữ liệu được xử lý với hàm preprocess thứ nhất là ```data_clean()```

In [ ]:
### xây tập vocab và kiểm tra độ phủ của tập vocab với dữ liệu đã được xử lý
total_text=pd.concat([df_train.question_text_cleaned,df_test.question_text_cleaned])
vocabulary=vocab_build(total_text)
oov=check_voc(vocabulary,model_embed) #oov: out of vocab

Với hàm preprocess thứ nhất là ```data_clean()``` thì thầy có thể thấy kết quả độ bao phủ đạt **81.04%**, hơi thấp một chút dù dữ liệu đã được xử lý từ trước<br>
Nhưng tệ hại là tập vocab chỉ bao phủ **25.40%** corpus<br>
**Nhận xét:**<br>
Có thể là do việc đụng chạm tới nhiều từ trong câu khi loại bỏ nó, và sửa lỗi sai chính tả. Điều này đã khiến giảm đi đáng kể số lượng các từ được tìm thấy trong file pretrained\
**Giải pháp:**<br>
Sử dụng một hàm Preprocess mới mà ít đụng chạm tới các từ, chỉ xử lý một cách đơn giản các câu

Trong block code này, em sẽ sử dụng hàm Preprocess thứ 2 (hàm process đơn giản hơn)

In [ ]:
df_test['question_text_cleaned_2'] = Preprocess(df_test['question_text'])
df_train['question_text_cleaned_2'] = Preprocess(df_train['question_text'])
total_text_2=pd.concat([df_train.question_text_cleaned_2,df_test.question_text_cleaned_2])
vocabulary2=vocab_build(total_text_2)
oov2=check_voc(vocabulary2, model_embed)

Như thầy có thể thấy hiệu quả được cải thiện một cách rõ rệt:
- Độ phủ của vocab với corpus tăng gấp **2.5 lần** lên **61.37%**
- Độ phủ tăng lên **90.81%**, một con số khá ấn tượng

Nếu như chuyển các từ về dạng ```lowercase``` thì 2 con số tương đương là **38.85%** với corpus và **89.47%** với vocab.<br>
Người Anh đặt cái tôi của họ rất cao, nên trong tất cả các câu văn của họ thì từ 'I' (chỉ bản thân người nói) luôn được đặt ở trạng thái in hoa ```uppercase```nên nếu chúng ta chuẩn hoá các từ về ```lowercase``` thì sẽ không lookup được từ đó trong file pretrained embeddings và sẽ gây ra hiện tượng thiếu sót trong ma trận embeddings mà chúng ta chuẩn bị xây dựng

In ra các từ và số lần xuất hiện trong Vocabulary<br>
Và do đó những câu mà với nhiều lần xuất hiện của các từ này có khả năng cao là các câu không đạt tiêu chuẩn

In [ ]:
sort_oov=dict(sorted(oov2.items(), key=operator.itemgetter(1),reverse=True))
dict(list(sort_oov.items())[:50])

Xoá các biến không dùng tới

In [ ]:
del oov, oov2,sort_oov,total_text,total_text_2
gc.collect()

Lấy ra index của từ trong tập từ điển phục vụ cho mã hóa onehot<br>
Ở dưới đây em sẽ định nghĩa 2 hàm là:
- ```get_word_index()```: lấy ra index của words trong tập từ vựng
- ```fit_one_hot()```: mã hoá word_index sang dạng onehot dựa vào corpus

In [ ]:
def get_word_index(vocab):
    word_index=dict((w,i+1) for i,w in enumerate(vocab.keys()))
    return word_index
def fit_one_hot(word_index,corpus):
    sent=[]
    for text in tqdm(corpus):
        li=[]
        for word in text.split():
            try:
                li.append(word_index[word])
            except KeyError:
                li.append(0)
        sent.append(li)
    return sent

Từ word_index ánh xạ sang tập train và encode sang mã hóa onehot\
Chắc chắn không thể tránh khỏi việc các chuỗi có độ dài không bằng nhau nên việc cần phải làm là padding tất cả các chuỗi\
Cắt đi các chuỗi có độ dài lớn hơn 40 và bù số 0 và những chuỗi có độ dài nhỏ hơn 40

In [ ]:
train,val=train_test_split(df_train,test_size=0.2,stratify=df_train.target,random_state=123)
vocab_size=len(vocabulary2)+1
max_len=40

word_index=get_word_index(vocabulary2)
### Chuẩn bị dữ liệu đã được xử lý
train_text=train['question_text_cleaned_2']
val_text=val['question_text_cleaned_2']
test_text=df_test['question_text_cleaned_2']

### mã hóa câu trong tập train sang dạng onehot cho dễ xử lý
encodes=fit_one_hot(word_index,train_text)
train_padded=pad_sequences(encodes,maxlen=max_len,padding="post")

### mã hóa câu trong tập validation sang dạng onehot cho dễ xử lý
encodes_=fit_one_hot(word_index,val_text)
val_padded=pad_sequences(encodes_,maxlen=max_len,padding="post")

### mã hóa câu trong tập test sang dạng onehot cho dễ xử lý
encodes__=fit_one_hot(word_index,test_text)
test_padded=pad_sequences(encodes__,maxlen=max_len,padding="post")

Xây dựng ma trận embeddings dựa trên tập từ vựng có trước ở trong file embeddings<br>
Ở đây mỗi hàng sẽ có vector embeddings cho mỗi từ duy nhất

In [ ]:
count=0
embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=model_embed[word]
        embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

Như có thể thấy đối với tập pretrained **Google News** dù độ phủ khá cao nhưng số lượng từ nằm ngoài tập embeddings khá là lớn khi lên tới 99931 từ<br>
Và cần phải có sự cải thiện về số lượng từ ```out_of_vocab``` nếu không thì hiệu quả của model sẽ kém.

# **4. Chuẩn bị Model**

Chuẩn bị model để train, ở đây em sử dụng Keras một phần vì nó có thể sử dụng với GPU và TPU<br>
Bên cạnh đó Keras hỗ trợ build model LSTM:

- LSTM là một mạng cải tiến của RNN nhằm giải quyết các vấn đề nhớ các bước dài của RNN (Mạng RNN chứa các vòng lặp bên trong cho phép thông tin có thể lưu lại được nhằm giải quyết vấn đề nhớ thông tin của mạng nơ ron truyển thống)
- Có thể coi LSTM là một dạng đặc biết của mạng nơ ron hồi quy, nó có khả năng học được các phụ thuộc xa 
- LSTM được thiết kế để tránh được vấn đề phụ thuộc xa (long-term dependency). Việc nhớ thông tin trong suốt thời gian dài là đặc tính mặc định của chúng, chứ ta không cần phải huấn luyện nó để có thể nhớ được. Tức là ngay nội tại của nó đã có thể ghi nhớ được mà không cần bất kì can thiệp nào.

**Mô hình RNN**
![rnn](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png)

> Mọi mạng hồi quy đều có dạng là một chuỗi các mô-đun lặp đi lặp lại của mạng nơ-ron. Với mạng RNN chuẩn, các mô-dun này có cấu trúc rất đơn giản, thường là một tầng tanh

**Mô hình LSTM**
![lstm](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png)

> LSTM cũng có kiến trúc dạng chuỗi như vậy, nhưng các mô-đun trong nó có cấu trúc khác với mạng RNN chuẩn. Thay vì chỉ có một tầng mạng nơ-ron, chúng có tới 4 tầng tương tác với nhau một cách rất đặc biệt

Sử dụng embedding layer, mục đích là để embedding sang một không gian mới có chiều nhỏ hơn, giảm chiều dữ liệu<br>
Bidirectional(LSTM) để xây model LSTM<br>
LSTM cũng là mạng CNN nên cần qua 2 lớp là Convo1D và Pool1D (convolution và pooling)

In [ ]:
def get_model_origin(matrix):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size,300,weights=[matrix],input_length=max_len,trainable=False)(inp)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Conv1D(64,3,activation="relu")(x)
    x = GlobalMaxPool1D()(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    return model

Sau khi build model ta được:<br>
&nbsp;<br>
![](https://scontent-hkt1-1.xx.fbcdn.net/v/t1.15752-9/188040073_326765032379284_902345591056315145_n.png?_nc_cat=101&ccb=1-3&_nc_sid=ae9488&_nc_ohc=4BVyEHYLeQ4AX9of6VB&_nc_ht=scontent-hkt1-1.xx&oh=dc02410144f78632c7701dcc691dac97&oe=60DD8312)

Gói việc tạo model vào một hàm ném vào **```stategy.scope()```** để enable khả năng chạy với TPU, tăng tốc độ train

In [ ]:
opt=Adam(learning_rate=0.001)
BATCH_SIZE = 1024
bin_loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0, name='binary_crossentropy')

### Xác định điểm callback để giảm learning rate, và restore lại trọng số tốt nhất kề trước 
early_stopping=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,mode="min",restore_best_weights=True)

### Giảm learning rate khi model không được cải thiên (càng học càng ngu)
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau( monitor="val_loss", factor=0.2, patience=2, verbose=1, mode="auto")

my_callbacks=[early_stopping,reduce_lr]

Ở trong hàm này em sử dụng Optimizer là ```Adam```. Về cơ bản ```Adam``` là sự kết hợp của ```Momentum``` và ```RMSprop```. Em sẽ không đi sâu vào 2 optimizer ```Momentum``` và ```RMSprop```.<br>
Có thể hiểu ```Momentum``` là một quả cầu lao xuống dốc theo trọng lực, còn ```Adam``` như một quả cầu sắt rất nặng và có ma sát.<br>
Giống với ```Adadelta``` và ```RMSprop```, nó duy trì trung bình bình phương độ dốc (slope) quá khứ vt và cũng đồng thời duy trì trung bình độ dốc quá khứ mt, giống ```Momentum```.

<p align="center">
  <img width="300" src="https://images.viblo.asia/1848e210-757a-4fd3-a662-2834b9c68f45.png">
</p>

Với công thức update:
$$g_n \leftarrow \nabla f(\theta_{n-1})$$
$$m_n \leftarrow (\frac{\beta_1}{1-\beta_1^{n}}).m_{n-1} + (\frac{1 - \beta_1}{1-\beta_1^{n}}).g_n$$
$$v_n \leftarrow (\frac{\beta_2}{1-\beta_2^{n}}).v_{n-1} + (\frac{1 - \beta_2}{1-\beta_2^{n}}).g_n \odot g_n$$
$$\theta_n \leftarrow \theta_{n-1} - \frac{a.m_n}{\sqrt{v_n} + \epsilon}$$

Hiểu một cách ngắn gọn thì ```Adam``` là optimizer tốt nhất hiện nay

Đánh giá độ mất mát thông tin dựa trên metric ```binary_cross_entropy```

Trong quá trình model học, nếu ta fix cứng ```epoch``` thì nhiều khả năng sẽ gặp tình trạng khi model đạt một ngưỡng đủ tốt sẽ bắt đầu quá trình học sai (tức càng học thì độ hiệu quả càng giảm). Nên ở đây em sẽ tạo một hàm callbacks để phục hồi lại trọng số tốt nhất kề trước mà model học được 

Và em sẽ giảm learning rate mỗi lần model đi qua một ```epoch``` để tăng độ hiệu quả của mô hình, với mỗi lần giảm 90%<br>
Tức là: $$LR = LR*factor$$

Chạy chay thì mô hình này tốn khá nhiều thời gian nên em đã sử dụng TPU cho nó nhanh hơn tí 🤦‍♀️

In [ ]:
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

# **5. Huấn luyện và Dự đoán**

### **a. Thử nghiệm model 1**

Compile model với 30 epoch cho máu, batch_size để ở mức cao để tận dụng sức mạnh tính toán của TPU và sử dụng với tập validation để validate dữ liệu, truyền vào tham số callback để mô hình càng học càng khôn chứ không được ngu đi<br>
Một phần do em có cậu bạn cũng train với bọn Keras nhưng khi f1 score đạt được tốt rồi thì nó bắt đầu triệu chứng ngu đi khi mà loss tằng (overfitting)

In [ ]:
with strategy.scope():
    google_model = get_model_origin(embedding_mat)
    google_model.compile(loss=bin_loss, optimizer=opt, metrics=['accuracy'])
history=google_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

In ra f1 score, ```f1_score``` tốt nhất ở đây là ```0.673``` tức ```67.3%``` ứng với hàm Preprocess thứ 2 tức là không đụng chạm nhiều đến câu hỏi<br>
Tương ứng là f1_score = ```0.633``` tức ```63.3%``` đối với hàm Preprocess đầu tiên khi loại bỏ ```stop_words```, ```lemma_words```, ...
Có thể nhận xét rằng:
- Với cách tiếp cận dựa trên pretrained thì việc tiền xử lý dữ liệu quá nhiều thực sự có ảnh hưởng lớn tới hiệu năng của model khi mà nó làm mất đi phần nhiều ý nghĩa các các từ có trong câu hỏi
- Có thể do cách cài đặt khác nhau nhưng về cơ bản với cách tiếp cận dựa vào pretrained thì ta không nên xử lý nhiễu của dữ liệu nhiều, có chăng là loại bỏ đi các digits, các công thức toán và các đường dẫn liên kết 

In [ ]:
# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Có thể thấy được rằng mô hình hoạt động khá hiệu quả với tập **```train```** khi mà ```loss``` giảm đều và ```accuracy``` tăng đều qua các epoch<br>
Tuy nhiên thì với tập **```test```** lại hoạt động không được hiệu quả khi ```loss``` và ```accuracy``` có hình dạng trồi sụt nhìn như sóng biển. Và tệ hơn là với tập **```test```** thì ```loss``` về các epoch cuối lại tăng khá cao.<br>
Vậy thì có thể kết luận là do **model** đã xây dựng **hoạt động chưa hiệu quả**, cần phải xem xét lại<br>

Lấy ra ```thresh_hold``` cho ```result``` tốt nhất và sử dụng nó để submit lên Kaggle

In [ ]:
google_y_pre=google_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(google_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

Như thầy có thể thấy thì kết quả tốt nhất loanh quanh ở ngưỡng tối đa **```0.67x```**<br> (x>3)
Ta sẽ sử dụng ngưỡng trung bình là **```0.675```** này để tham chiếu với các ma trận embeddings được xây dựng với 3 anh còn lại.

In [ ]:
del model_embed, history, best_score, best_thresh, google_model, google_y_pre
gc.collect()

### **b. Thử nghiệm model 2**

Bây giờ em sẽ thử nghiệm một model mới, dựa trên nguồn tham khảo từ tác giả **[taziz437](https://www.kaggle.com/taziz437)** với bài dự thi [TariqAziz](https://www.kaggle.com/taziz437/tariqaziz) <br>
Với **model** cũ tức ```get_model_origin``` thì em đã build model với mô hình **LSTM** 128 chiều<br>
Còn với tác giả **[taziz437](https://www.kaggle.com/taziz437)** anh ấy đã kết hợp mô hình **LSTM** 256 chiều và **GRU** 128 chiều để tạo model với 2 lớp **Pooling1D**<br>
Theo cảm quan đánh giá thì có vẻ **model** này sẽ cho một hiệu năng tốt hơn và ổn định hơn so với **model** mà em đã tự tạo ở trên.<br>
Dông dài thì cũng đến thế, em sẽ thử nghiệm model mới ở ngay bên dưới.

In [ ]:
def get_model(matrix):
    inp = Input(shape=(max_len,))
    x = Embedding(vocab_size, 300, weights=[matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.3)(x)
    x1 = Bidirectional(LSTM(256, return_sequences=True))(x)
    x2 = Bidirectional(GRU(128, return_sequences=True))(x1)
    max_pool1 = GlobalMaxPooling1D()(x1)
    max_pool2 = GlobalMaxPooling1D()(x2)
    conc = Concatenate()([max_pool1, max_pool2])
    predictions = Dense(1, activation='sigmoid')(conc)
    model = Model(inputs=inp, outputs=predictions)
    adam = optimizers.Adam(lr=0.001)
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    return model

Model được thiết kế với cùng ```optimizer``` là ```Adam``` với ```learning_rate=0.001``` và với metric ```accuracy``` và hàm ```loss``` được đánh giá dựa trên ```binary_crossentropy```

In [ ]:
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

Nếu không khởi tạo lại ```strategy.scope()``` thì em bị gặp lỗi là ```loss``` của model mới sẽ cao hơn mức bình thường dẫn đến việc ```model``` phải chạy qua nhiều epoch hơn và tốn thời gian để train hơn<br>
Nên em phải thực hiện việc init lại ```strategy.scope()``` mỗi lần tạo và compile một model mới.

In [ ]:
with strategy.scope():
    google_model = get_model(embedding_mat)
    google_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
history=google_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

Có thể thấy rằng model mới hoạt động hiệu quả hơn hẳn model cũ khi mà nó đã đẩy được ```f1_score``` lên thêm 0.01 tức 1% từ mức ```0.67``` làm cơ sở lên mức ```0.68x``` (x bé tí teo).<br>
Tuy nhiên đây được coi là mức cải thiện khá tốt so với model cũ, ```loss``` của tập validate cũng nhỏ hơn

In [ ]:
# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Thấy được một điều rõ ràng là ```loss``` của tập validate đã giảm đi và có hình dạng ổn định hơn so với model cũ khi mà về những ```epoch``` cuối thì loss không tăng nhiều như model cũ.<br>
```accuracy``` cũng được cải thiện hơn phần nào và cũng ổn định hơn một chút, ở mức cao

In [ ]:
google_y_pre=google_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(google_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(google_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

Model cũ khá chật vật để đạt mức maximum là ```0.675``` thì model mới dễ dàng đạt được mức trên ```0.68``` của ```f1_score``` và xấp xỉ ở ngưỡng ```0.69```<br>
Vậy có thể yên tâm sử dụng model mới bởi vì hiệu suất và độ ổn định của nó

In [ ]:
threshold=best_thresh
google_y_test_pre=google_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
google_y_test_pre=(google_y_test_pre>thresh).astype(int)

In [ ]:
del embedding_mat, history, best_score, best_thresh
gc.collect()

# **6. So sánh, mở rộng**
Do sử dụng 1 pretrained là **Google News** nên kết quả thu được ở mức là **```0.68x``` (x<=3)** f1_score<br>
&nbsp;<br>
Bởi vì 3 thằng còn lại (không phải **Google News**) đều không ở dạng binary và không phải ở dạng chuẩn word_vector nên không đọc được bằng KeyedVectors, em sẽ định nghĩa một hàm để đọc các file embeddings còn lại <br>
Tuy nhiên do tự định nghĩa nên thời gian load dữ liệu vào biến khá là lâu

#### **Reference:** https://www.kaggle.com/theoviel/improve-your-score-with-some-text-preprocessing

Do có thằng ```wiki``` là ngoại đạo với đuôi là ```.vec``` nên cần định nghĩa riêng cho nó 1 ```statement```

In [ ]:
def load_embed(file):
    def get_coefs(word,*arr):
        return word, np.asarray(arr, dtype='float32')

    if file == '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec':
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file) if len(o)>100)
    else:
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(file, encoding='latin'))

    return embeddings_index

In [ ]:
!tree -h ./

In [ ]:
glove_path = './glove.840B.300d/glove.840B.300d.txt'
paragram_path = './paragram_300_sl999/paragram_300_sl999.txt'
wiki_path = './wiki-news-300d-1M/wiki-news-300d-1M.vec'

Load các file embeddings vào biến<br>
Tuy nhiên do load thủ công không qua thư viện nên việc load khá là chậm, mất khoảng 9' để load cả 3 anh<br>
Bọn ```KeyedVector``` có hỗ trợ load ```glove``` nhưng phải covert qua binary nên về tốc độ thì cũng không hơn anh thủ công là mấy

In [ ]:
%%time
glove_embed = load_embed(glove_path)
print("\033[1mGlove Coverage: \033[0m]")
oov_glove = check_voc(vocabulary2, glove_embed)

Có thể dễ dàng nhận thấy anh **Google News** cho độ phủ không tốt bằng anh **Glove** khi anh **Glove** đạt độ phủ trên **99.39%** và **72.81%** với corpus qua đó có thể đoán được rằng ma trận embeddings của anh **Glove** sẽ giúp cho mô hình đạt được hiệu suất cao hơn

### **a. Glove**

In [ ]:
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

Lần đầu chạy không định nghĩa lại ```strategy.scope()``` thì em gặp một hiện tượng lạ đó là với việc build model mới và chạy trong ```strategy.scope()``` thì model có vẻ bị sai khá nhiều khi ```loss``` lên tới mức ```0.22``` với ```compile``` lần 1 và ```loss``` ở mức ```0.27``` với lần ```compile``` thứ 2<br>
Do vậy em đoán khả năng là do thằng ```strategy.scope()``` giữ lại một số giá trị gì đó nên em đã định nghĩa lại thằng ```strategy.scope()```

In [ ]:
count=0
glove_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=glove_embed[word]
        glove_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

Có thể thấy được rằng số từ ```out_of_vocab``` giảm đi khá nhiều, ở mức 70336 từ, và qua đó có thể kỳ vọng mô hình đem lại độ chính xác cao hơn.<br>
Và em vẫn chưa có cách để xử lý những từ ```out_of_vocab``` như này 🥺🥺🥺

In [ ]:
with strategy.scope():
    glove_model = get_model(glove_embedding_mat)
    glove_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
glove_history=glove_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

Khi truyền optimizer bằng biến ```opt``` cũ thì đã xảy ra lỗi và bắt buộc phải tự pass lại metrics mới vào trong ```model.compile()``` do đó em đã truyền vào optimizer là ```Adam``` với ```learning_rate``` bằng ```0.001``` giống ở trên để đảm bảo sự công bằng cho thử nghiệm

In [ ]:
# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(glove_history.history['loss'])
plt.plot(glove_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(glove_history.history['accuracy'])
plt.plot(glove_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Khi sử dụng mô hình với ma trận embeddings sinh ra từ **Glove** thì nó cũng cho ra một kết quả khá giống với **Google News** khi mà **```train```** thì mô hình hoạt động khá hiệu quả <br>
Tuy nhiên cũng như **Google News** thì đối với tập test, càng về epoch cuối thì ```loss``` càng tăng cao, tuy nhiên có sự cải thiện hơn **Google News** khi mà ```accuracy``` có hình dạng ổn định hơn một chút.

In [ ]:
glove_y_pre=glove_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(glove_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(glove_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(glove_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

Rõ ràng khi sử dụng với file embeddings **Glove** thì kết quả đã có sự cải thiện với ```f1_score``` loanh quanh **```0.69```** tức **69%**<br>
Tuy nhiên sự cải thiện là không nhiều <br>
Điều này có thể kết luận rằng số từ ```out_of_vocab``` và độ phủ của vocab thực sự ảnh hưởng tới hiệu năng của model.<br>

In [ ]:
threshold=best_thresh
glove_y_test_pre=glove_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
glove_y_test_pre=(glove_y_test_pre>thresh).astype(int)

In [ ]:
del glove_embed, glove_embedding_mat, glove_history, best_score, best_thresh
gc.collect()

### **b. Paragram**

In [ ]:
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

In [ ]:
%%time
paragram_embed = load_embed(paragram_path)
print("\033[1mParagram Coverage: \033[0m]")
oov_paragram = check_voc(vocabulary2, paragram_embed)

Có thể thấy rằng độ phủ của anh **Paragram** còn tốt hơn anh **Glove** khi đạt tới **81.11%** và độ phủ với ```corpus``` đạt tới **38.97%**.<br> Kì vọng là sẽ tạo ra ma trận embeddings giúp cho model có hiệu năng tệ nhất

In [ ]:
count=0
para_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=paragram_embed[word.lower()]
        para_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

So với 2 file embeddings ở trên là **Google News** và **Glove** thì số lượng từ nằm ngoài tập vocab ```out_of_vocab``` tăng lên khá nhiều khi đạt con số 157867 từ nếu không chuẩn hoá hết sang dạng lowercase<br>
Bên cạnh đó thì **Paragram** cho độ phủ tệ nhất với **81.11%**. nếu không chuẩn hoá sang lowercase<br>
Có thể kỳ vọng với ma trận embeddings sinh ra từ **Paragram** thì mô hình sẽ đạt hiệu quả tệ nhất.<br>
Tuy nhiên khi chuẩn hoá sang lowercase và lookup thì độ phủ của **Paragram** tăng lên rõ rệt và có số từ ```out_of_vocab``` giảm mạnh xuống còn 61842 từ, do vậy ma trận embeddings sẽ giúp model đạt hiệu quả cao hơn

In [ ]:
with strategy.scope():
    para_model = get_model(para_embedding_mat)
    para_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
para_history=para_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

Ở ```epoch``` cuối cùng thì đúng như dự đoán khi mà ```loss``` của mô hình đi kèm với ma trận embeddings của **Paragram** đạt được ```loss``` ở mức thấp nhất là ```0.5x``` (x học tiểu học) và đi kèm với đó là ```accuracy``` cao đạt mức ```0.9791```.<br>
Tuy nhiên điều quan trọng hơn đó chính là đánh giá hiệu năng của mô hình thông qua ```f1_score```

In [ ]:
# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(para_history.history['loss'])
plt.plot(para_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(para_history.history['accuracy'])
plt.plot(para_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Cũng như 2 anh ở trên thì mô hình với ma trận embeddings của **Paragram** lại cho ```loss``` của tập **```test```** tăng dần về những epoch cuối.<br>
Tuy nhiên đây vẫn là kết quả chấp nhận được do sự chênh lệch không quá nhiều

In [ ]:
para_y_pre=para_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(para_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(para_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(para_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

Qua nhiều lần thử nghiệm có thể thấy rằng anh **Paragram** cho hiệu năng không như kì vọng khi anh ấy đạt độ phủ tốt nhất, oov thấp nhất **(khi chuẩn hoá text về ```lowercase```)**, tuy nhiên model khi train với ma trận embeddings của anh **Paragram** cho hiệu năng vẫn kém hơn anh **Glove**

In [ ]:
threshold=best_thresh
para_y_test_pre=para_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
para_y_test_pre=(para_y_test_pre>thresh).astype(int)

In [ ]:
del paragram_embed, para_embedding_mat, para_history, best_score, best_thresh
gc.collect()

### **c. Wiki**

In [ ]:
strategy = None

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
    strategy = tf.distribute.TPUStrategy(tpu)
    print('Use TPU')
except ValueError:
    if len(tf.config.list_physical_devices('GPU')) > 0:
        strategy = tf.distribute.MirroredStrategy()
        print('Use GPU')
    else:
        strategy = tf.distribute.get_strategy()
        print('Use CPU')

In [ ]:
%%time
wiki_embed = load_embed(wiki_path)
print("\033[1mWiki Coverage: \033[0m]")
oov_wiki = check_voc(vocabulary2, wiki_embed)

So với 2 anh **Glove** thì độ phủ của anh **Wiki** thấp hơn một chút, tuy nhiên qua ví dụ của anh **Paragram** thì không nói trước được điều gì

In [ ]:
count=0
wiki_embedding_mat=np.zeros((vocab_size,300))
for word,i in tqdm(word_index.items()):
    try:
        vec=wiki_embed[word]
        wiki_embedding_mat[i]=vec
    except KeyError:
        count+=1
        continue

print("Number of Out of Vocabulary",count)

Số lượng từ của anh **Wiki** cũng ở mức khá cao khi gần tiệm cận với anh **Google News**, vậy có thể là ma trận embeddings sẽ có nhiều từ không tìm được và hiệu năng model cũng sẽ không tốt bằng anh **Glove**

In [ ]:
with strategy.scope():
    wiki_model = get_model(wiki_embedding_mat)
    wiki_model.compile(loss=bin_loss, optimizer=Adam(lr=0.001), metrics=['accuracy'])
wiki_history=wiki_model.fit(train_padded, train.target, batch_size=BATCH_SIZE, epochs=30, validation_data=(val_padded, val.target),callbacks=my_callbacks)

In [ ]:
# summarize history for loss
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 1)
plt.plot(wiki_history.history['loss'])
plt.plot(wiki_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# summarize history for accuracy
plt.figure(figsize=(15, 5)).add_subplot(1, 2, 2)
plt.plot(wiki_history.history['accuracy'])
plt.plot(wiki_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Cũng như 3 anh trên là **Glove**, **Paragram** và **Google News** thì anh **Wiki** này cũng có ```loss``` của tập test tăng dần về epoch cuối và ```accuracy``` không được cải thiện nhiều.<br>
Vậy nên ta cần phải config lại model, để đạt được hiệu quả cao hơn

In [ ]:
wiki_y_pre=wiki_model.predict(val_padded, verbose=1)
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(wiki_y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(wiki_y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(wiki_y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

Tuy độ phủ không đạt được như anh **Paragram** và số lượng từ ```out_of_vocab``` xấp xỉ anh **Google News** nhưng anh **Wiki** này cho hiệu năng rất đáng nể khi mà ma trận embeddings của anh này giúp model đạt ```f1_score``` xấp xỉ anh **Glove** tuy rằng **Glove** vẫn có phần nhỉnh hơn

In [ ]:
threshold=best_thresh
wiki_y_test_pre=wiki_model.predict(test_padded, batch_size=BATCH_SIZE, verbose=1)
wiki_y_test_pre=(wiki_y_test_pre>thresh).astype(int)

In [ ]:
del wiki_embed, wiki_embedding_mat, wiki_history, best_score, best_thresh
gc.collect()

Oke vậy thì có thể thấy được rằng hiệu năng của model với các ma trận embeddings đến từ vị trí của 4 anh là **Google News**, **Glove**, **Paragram** và **Wiki** khá tương đồng với nhau, duy chỉ có anh **Google News** là thể hiện kém hơn 3 anh còn lại, cả về độ phủ, cả về ```f1_score``` và những yếu tố khác.<br>
Tuy nhiên chúng ta lại có thể thấy được rằng, với cả 4 model này tại sao lại không kếp hợp chúng vào với nhau để tạo ra một ```predict``` hiệu quả, thì dưới đây chính là sự kết hợp của cả 4 models<br>
Thuật ngữ chuyên ngành gọi là **Stack Models Prediction**<br>
Với **Stack Models Prediction** thì chúng ta có nhiều chiến lược, nhưng ở đây em sẽ nhân mỗi thằng ```predict``` với cùng một hệ số.<br>
Do có 4 thằng nên hệ số cho mỗi thằng là ```0.25``` và ta được một ```predict``` mới.<br>
Và việc ghép các **predicts** lại hoạt động khá hiệu quả khi ```f1_score``` của mô hình đạt tới **70%** tức ```0.70```

In [ ]:
y_pre=0.25*(google_y_pre + glove_y_pre + para_y_pre + wiki_y_pre)
# y_pre=0.20*google_y_pre + 0.35*glove_y_pre + 0.15*para_y_pre + 0.30*wiki_y_pre
best_score = 0
best_thresh = 0
for thresh in np.arange(0.1,0.5,0.01):
    if(best_score < metrics.f1_score(val.target,(y_pre>thresh).astype(int))):
        best_score = metrics.f1_score(val.target,(y_pre>thresh))
        best_thresh = round(thresh, 2)
    print("threshold {0:2.2f} f1 score:{1:2.3f}".format(thresh,metrics.f1_score(val.target,(y_pre>thresh).astype(int))))
print("\033[1mBest result {0:2.3f} in thresh_hold {1:2.2f}\033[0m".format(best_score, best_thresh))

In [ ]:
# y_test_pre = 0.25 * (google_y_test_pre + glove_y_test_pre + para_y_test_pre + wiki_y_test_pre)
y_test_pre = 0.2*google_y_test_pre + 0.3*glove_y_test_pre + 0.2*para_y_test_pre + 0.3*wiki_y_test_pre
y_test_pre=(y_test_pre>thresh).astype(int)
### Tạo File submission
submit=pd.DataFrame()
submit["qid"]=df_test.qid
submit["prediction"]=y_test_pre
submit.to_csv("submission.csv",index=False)
print("ok")